In [1]:
import dpkt
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import os
from BfTableGenerator import TDmapLoader
from tqdm import tqdm
from Utils import *
from DDBSCAN import Raster_DBSCAN
import open3d as op3
from open3d.web_visualizer import draw
from VisulizerTools import *
import time
from sklearn.cluster import DBSCAN
import cv2 as cv

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [15]:
file_path = r'D:\LiDAR_Data\MidTown\Thoma\2021-12-18-12-0-0.pcap'
loader = TDmapLoader(file_path).frame_gen()

In [16]:
threshold_map = np.load(r'D:\LiDAR_Data\MidTown\Thoma\Calibration\bck_map.npy')

In [17]:
clustering_td = Raster_DBSCAN(window_size=(5,15),eps = 1.2, min_samples= 12, Td_map_szie=threshold_map.shape)   

In [18]:
Td_maps = []
for td in tqdm(loader):
    if td is not None:
        Td_maps.append(td)
    else:
        break

65it [00:02, 28.12it/s]


KeyboardInterrupt: 

In [6]:
def get_maps(db,Td_map,threshold_map):
    Foreground_map = (Td_map < threshold_map)&(Td_map != 0)
    Labeling_map = clustering_td.fit_predict(Td_map= Td_map,Foreground_map=Foreground_map)
    return Foreground_map,Labeling_map

In [7]:
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

In [8]:
Foreground_map0,Labeling_map0 = get_maps(clustering_td,Td_maps[0],threshold_map)
Foreground_map1,Labeling_map1 = get_maps(clustering_td,Td_maps[1],threshold_map)

In [56]:
test = Td_maps[20].copy()
test[test>220] = 220
test = ((test/220)*255).astype(np.uint8)
mask0 = np.zeros_like(Td_maps[0])
mask0[Foreground_map0] = 255
mask0 = mask0.astype(np.uint8)

In [60]:
p0 = cv2.goodFeaturesToTrack(test, mask = None, **feature_params)

In [61]:
p0

array([[[  39.,   18.]],

       [[  27.,   20.]],

       [[ 990.,   26.]],

       [[  93.,   23.]],

       [[1781.,   24.]],

       [[  81.,   22.]]], dtype=float32)

In [ ]:
flow = cv.calcOpticalFlowFarneback(prev_gray, gray, None,0.5, 3, 15, 3, 5, 1.2, 0)

In [163]:
clustering_td = Raster_DBSCAN(window_size=(5,15),eps = 1.2, min_samples= 12, Td_map_szie=threshold_map.shape)   
Td_map = Td_maps[-1]
Foreground_map = (Td_map < threshold_map)&(Td_map != 0)
Labeling_map = clustering_td.fit_predict(Td_map= Td_map,Foreground_map=Foreground_map)
Background_map = (Td_map >= threshold_map)&(Td_map != 0)
pcd = get_pcd_colored_filtered(Td_map,Labeling_map,Foreground_map)
draw(pcd,height=1000,width=1000)

WebVisualizer(window_uid='window_1')

In [52]:
clustering_td = Raster_DBSCAN(window_size=(5,15),eps = 1.2, min_samples= 12, Td_map_szie=threshold_map.shape)   
Td_map = Td_maps[20]
Foreground_map = (Td_map < threshold_map)&(Td_map != 0)
Labeling_map = clustering_td.fit_predict(Td_map= Td_map,Foreground_map=Foreground_map)
Background_map = (Td_map >= threshold_map)&(Td_map != 0)
pcd = get_pcd_colored_filtered(Td_map,Labeling_map,Foreground_map)
draw(pcd,height=1000,width=1000)

WebVisualizer(window_uid='window_1')

In [50]:
Foreground_map = np.full(Td_maps[0].shape,True)
Labeling_map = np.ones_like(Foreground_map)
pcd = get_pcd_colored(Td_maps[0],Labeling_map)

In [51]:
draw(pcd)

WebVisualizer(window_uid='window_0')